In [239]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math

In [240]:
# def makeDataFrame(filename):
#     data=pd.read_csv(filename)
#     df=pd.DataFrame(data,columns=['Adj Close'])
#     print(df)
#     s=df.values.tolist()
#     return s 

makeDataFrame(filename)

In [241]:
# parameters
S0=30
r=0.06
t=4
rate_drift = 0
rate_volatility = 0.005
vol=5 # volatility for stock price: for additive amount, how much to add
drift = 0 # drift for stock price
u=1+r+math.sqrt((1+r)**2-1) 
d=(1+r)/u
face=50
coupon_rate = 0.04
coupon = coupon_rate*face
default_threshold = 1/3
bankrupcy_ratio = 2.4
K=55 # call at this value
Pu=50 # put at this value

In [242]:
def g(x): # payoff function e.g for a call it would be max(0, x-K)
    if x==0:
        return [0, 0]
    if x <= default_threshold*S0:
        return [x*bankrupcy_ratio, 0]
    return [face, coupon] # first entry is the actual value, second is dividends or coupons

In [243]:
p=0.5#(1+r-d)/(u-d)
q=1-p
p=1-q # fixes floating-point errors hopefully
print(p, q) # only use these in the multiplicative model

0.5 0.5


In [244]:
def nSigFigs(num, n):
    #print(num == np.nan)
    if type(num) not in [float, int, np.float64] or num == float("NaN"): return float("NaN")
    return int(num * 10**n) / 10**n

In [245]:
def displayChart(tab):
    Chart=[]
    columns = []
    N = len(tab)-1
    for i in range(N, -1, -1):
        chart = []
        columns.append(N-i)
        for j in range(0,i):
            chart.append("")
        for j in range(i, N+1):
            entry = tab[j][i]
            chart.append(entry)
        Chart.append(chart)
    df = pd.DataFrame(Chart, columns = columns)
    return df.style.hide_index()

In [246]:
def r_values(drift, vol):
    R = {}
    for i in range(t, -1, -1):
        R[i] = r + i*drift - vol*np.arange(i, -1, -1) + vol*np.arange(0, i+1, 1)
    return R

rates = r_values(rate_drift, rate_volatility)
print("interest rates")
displayChart(rates)

interest rates


0,1,2,3,4
,,,,0.080000
,,,0.075000,0.070000
,,0.070000,0.065000,0.060000
,0.065000,0.060000,0.055000,0.050000
0.060000,0.055000,0.050000,0.045000,0.040000


In [247]:
def StockPrice(mode='a'): # a for additive, m for multiplicative
    StockPrice={}
    if mode == 'm':
        for i in range(t, -1, -1):
            StockPrice[i] = S0 * d ** (np.arange(i,-1,-1)) * u ** (np.arange(0,i+1,1)) #np.empty(N+1)

    elif mode == 'a':
        for i in range(t, -1, -1):
            StockPrice[i] = np.maximum(np.zeros(i+1), S0 + i*drift - vol*np.arange(i, -1, -1) + vol*np.arange(0,i+1,1))
    '''for n in range(0, N+1):
        StockPrice[N].append(S0*(u**n)*(d**(N-n)))
    for i in range(N-1, -1, -1):
        StockPrice[i]=[] #np.empty(i+1)
        for j in range(0, i+1):
            StockPrice[i].append((StockPrice[i+1][j+1])/u)'''
    return StockPrice

SP = StockPrice()
print("stock price")
displayChart(SP)

# for i in range(t, -1, -1):
#     print(str(i) + ": "+str([nSigFigs(x, n=4) for x in SP[i]]))

stock price


0,1,2,3,4
,,,,50.000000
,,,45.000000,40.000000
,,40.000000,35.000000,30.000000
,35.000000,30.000000,25.000000,20.000000
30.000000,25.000000,20.000000,15.000000,10.000000


In [248]:
def p_values(stock_price_chart, rates):
    P = {i: [1]*(i+1) for i in range(t+1)}
    for i in range(0, t):
        for j in range(i+1):
            if stock_price_chart[i][j] == 0:
                P[i][j] = -1
            else:
                d = stock_price_chart[i+1][j] / stock_price_chart[i][j]
                u = stock_price_chart[i+1][j+1] / stock_price_chart[i][j]
                P[i][j] = (1 + rates[i][j] - d) / (u-d)
                # print(d)
                # print(u)
                # print(u-d)
                # print(1+rates[i][j])
                # print(1 + rates[i][j] - d)
    return P

p_vals = p_values(SP, rates)
print("p tilde")
displayChart(p_vals)

p tilde


0,1,2,3,4
,,,,1
,,,0.837500,1
,,0.780000,0.727500,1
,0.727500,0.680000,0.637500,1
0.680000,0.637500,0.600000,0.567500,1


In [249]:
# expected stock price (risk neutral pricing)
# additive model
def exp(x,y,p,r):
    if p != -1:
        return (x*p+y*(1-p))/(1+r)
    return 0

# multiplicative model
def rnExp(x,y):
    return exp(x,y,p,r)

In [250]:
bond_payoff = list(map(g, SP[t]))
def vanillaBond(payoff): # coupon amount
    vanillaBond={}
    vanillaBond[t]=payoff
    # IP=IntrinsicPut()
    #for n in range(0, N+1):
    #    PutPriceA[N].append(IP[N][n])
    for i in range(t-1, -1, -1):
        vanillaBond[i]=[]
        for j in range(i+1):
            entry = [0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(vanillaBond[i+1][j+1][0], vanillaBond[i+1][j][0], p_vals[i][j], rates[i][j]) + exp(vanillaBond[i+1][j+1][1], vanillaBond[i+1][j][1], p_vals[i][j], rates[i][j])
                entry[1] = coupon if (SP[i][j] >= default_threshold) and i>0 else 0
            #print(entry)
            vanillaBond[i].append(entry)
    return vanillaBond

vBond = vanillaBond(bond_payoff)
print("Vanilla Bond payoff: [face value (after coupon), coupon]")
displayChart(vBond)

Vanilla Bond payoff: [face value (after coupon), coupon]


0,1,2,3,4
,,,,"[50, 2.0]"
,,,"[48.37209302325582, 2.0]","[50, 2.0]"
,,"[47.17010896794226, 2.0]","[48.82629107981221, 2.0]","[50, 2.0]"
,"[46.40424372964024, 2.0]","[48.08904696472974, 2.0]","[49.28909952606635, 2.0]","[50, 2.0]"
"[45.6277702171945, 0]","[46.28297091897127, 2.0]","[44.61177070206753, 2.0]","[38.17224880382775, 2.0]","[24.0, 0]"


In [251]:
def convBond(payoff):
    convBond = {}
    convBond[t] = [[max(SP[t][i], payoff[i][0]), payoff[i][1]] for i in range(t+1)]
    for i in range(t-1, -1, -1):
        convBond[i]=[]
        for j in range(i+1):
            entry = [0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(convBond[i+1][j+1][0], convBond[i+1][j][0], p_vals[i][j], rates[i][j]) \
                         + exp(convBond[i+1][j+1][1], convBond[i+1][j][1], p_vals[i][j], rates[i][j])
                entry[0] = max(entry[0], SP[i][j])
                entry[1] = coupon if SP[i][j] >= default_threshold else 0
            #print(entry)
            convBond[i].append(entry)
    convBond[0][0][1]=0
    return convBond

cBond = convBond(bond_payoff)
print("Convertible Bond payoff: [face value (after coupon), coupon]")
print("Convertible but not callable")
displayChart(cBond)

Convertible Bond payoff: [face value (after coupon), coupon]
Convertible but not callable


0,1,2,3,4
,,,,"[50.0, 2.0]"
,,,"[48.37209302325582, 2.0]","[50, 2.0]"
,,"[47.17010896794226, 2.0]","[48.82629107981221, 2.0]","[50, 2.0]"
,"[46.40424372964024, 2.0]","[48.08904696472974, 2.0]","[49.28909952606635, 2.0]","[50, 2.0]"
"[45.6277702171945, 0]","[46.28297091897127, 2.0]","[44.61177070206753, 2.0]","[38.17224880382775, 2.0]","[24.0, 0]"


In [252]:
def callable2(convertible):
    callBond = {}
    callBond[t] = [[min(K, convertible[t][i][0]), convertible[t][i][1],0] for i in range(t+1)]
    for i in range(t+1):
        # 0 means not called, 1 means called
        callBond[t][i][2] = 1 if (K<=convertible[t][i][0]) else 0
    for i in range(t-1, -1, -1):
        callBond[i]=[]
        for j in range(i+1):
            entry = [0,0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(callBond[i+1][j+1][0], callBond[i+1][j][0], p_vals[i][j], rates[i][j])
                entry[0] += exp(callBond[i+1][j+1][1], callBond[i+1][j][1], p_vals[i][j], rates[i][j])
                entry[2] = 1 if (K<entry[0]) else 0
                entry[0] = min(max(K, SP[i][j]), entry[0])
                entry[1] = coupon if ((SP[i][j] >= default_threshold) and i>0) else 0
            #print(entry)
            callBond[i].append(entry)
    return callBond

callable_bond = callable2(cBond)
print("Callable Bond payoff: [face value (after coupon), coupon]")
print("Convertible and callable")
print("Conversion prioritized")
displayChart(callable_bond)

Callable Bond payoff: [face value (after coupon), coupon]
Convertible and callable
Conversion prioritized


0,1,2,3,4
,,,,"[50.0, 2.0, 0]"
,,,"[48.37209302325582, 2.0, 0]","[50, 2.0, 0]"
,,"[47.17010896794226, 2.0, 0]","[48.82629107981221, 2.0, 0]","[50, 2.0, 0]"
,"[46.40424372964024, 2.0, 0]","[48.08904696472974, 2.0, 0]","[49.28909952606635, 2.0, 0]","[50, 2.0, 0]"
"[45.6277702171945, 0, 0]","[46.28297091897127, 2.0, 0]","[44.61177070206753, 2.0, 0]","[38.17224880382775, 2.0, 0]","[24.0, 0, 0]"


In [253]:
def callable3(call_later):
    callBond = {}
    callBond[t] = [[max(SP[t][i], call_later[t][i][0]), call_later[t][i][1],0] for i in range(t+1)]
    for i in range(t+1):
        # 0 means not called, 1 means called
        callBond[t][i][2] = 1 if (K<=call_later[t][i][0]) else 0
    for i in range(t-1, -1, -1):
        callBond[i]=[]
        for j in range(i+1):
            entry = [0,0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(callBond[i+1][j+1][0], callBond[i+1][j][0], p_vals[i][j], rates[i][j])
                entry[0] += exp(callBond[i+1][j+1][1], callBond[i+1][j][1], p_vals[i][j], rates[i][j])
                entry[2] = 1 if (K<entry[0]) else 0
                entry[0] = max(SP[i][j], entry[0])
                entry[1] = coupon if ((SP[i][j] >= default_threshold) and i>0) else 0
            #print(entry)
            callBond[i].append(entry)
    return callBond

call_second = callable3(callable_bond)
print("Callable Bond payoff: [face value (after coupon), coupon]")
print("Convertible and callable")
print("Call before convertion")
displayChart(call_second)

Callable Bond payoff: [face value (after coupon), coupon]
Convertible and callable
Call before convertion


0,1,2,3,4
,,,,"[50.0, 2.0, 0]"
,,,"[48.37209302325582, 2.0, 0]","[50, 2.0, 0]"
,,"[47.17010896794226, 2.0, 0]","[48.82629107981221, 2.0, 0]","[50, 2.0, 0]"
,"[46.40424372964024, 2.0, 0]","[48.08904696472974, 2.0, 0]","[49.28909952606635, 2.0, 0]","[50, 2.0, 0]"
"[45.6277702171945, 0, 0]","[46.28297091897127, 2.0, 0]","[44.61177070206753, 2.0, 0]","[38.17224880382775, 2.0, 0]","[24.0, 0, 0]"


In [254]:
def putable(convertible):
    putBond = {}
    putBond[t] = [[convertible[t][i][0], convertible[t][i][1],0] for i in range(t+1)]
    for i in range(t+1):
        # 0 means not called, 1 means called
        putBond[t][i][2] = 1 if (K>=convertible[t][i][0]) else 0
    for i in range(t-1, -1, -1):
        putBond[i]=[]
        for j in range(i+1):
            entry = [0,0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(putBond[i+1][j+1][0], putBond[i+1][j][0], p_vals[i][j], rates[i][j])
                entry[0] += exp(putBond[i+1][j+1][1], putBond[i+1][j][1], p_vals[i][j], rates[i][j])
                if(SP[i][j]==0):
                    # bankrupt
                    entry[0] = 0
                    entry[1] = 0
                    entry[2] = 0
                elif(0<SP[i][j]<=default_threshold):  
                    # financial distress
                    entry[0] = max(Pu/2, entry[0])
                    entry[1] = 1 if ((Pu/2)>entry[0]) else 0
                    entry[2] = 0 
                else:       
                    entry[0] = max(Pu, entry[0]) if (i<t) else entry[0]
                    entry[2] = 1 if (Pu>entry[0]) else 0
                    entry[1] = coupon if (i>0) else 0
            #print(entry)
            putBond[i].append(entry)
    return putBond

put_bond = putable(cBond)
print("Putabl Bond payoff: [face value (after coupon), coupon]")
print("Convertible and putable")
displayChart(put_bond)

Putabl Bond payoff: [face value (after coupon), coupon]
Convertible and putable


0,1,2,3,4
,,,,"[50.0, 2.0, 1]"
,,,"[50, 2.0, 0]","[50, 2.0, 1]"
,,"[50, 2.0, 0]","[50, 2.0, 0]","[50, 2.0, 1]"
,"[50, 2.0, 0]","[50, 2.0, 0]","[50, 2.0, 0]","[50, 2.0, 1]"
"[50, 0, 0]","[50, 2.0, 0]","[50, 2.0, 0]","[50, 2.0, 0]","[24.0, 0, 1]"


In [255]:
#conv_payoff: dictionary of convertible prices
#strike: strike price
def fill_payoff(payoff, time):
    for i in range(time-1, -1, -1):
        payoff[i] = []
        for j in range(i+1):
            entry = [0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(payoff[i+1][j+1][0], payoff[i+1][j][0], p_vals[i][j], rates[i][j]) + exp(payoff[i+1][j+1][1], payoff[i+1][j][1], p_vals[i][j], rates[i][j])
                entry[0] = entry[0]
                entry[1] = coupon if (SP[i][j] >= default_threshold) and i>0 else 0
            payoff[i].append(entry)
    return payoff
        
        
def callable(conv, strike):
    callable_prices = []
    for i in range(t-1, -1, -1):    #i: time step
        for j in range(i+1):        #j: # of nodes at time
            if conv[i][j][0] >= strike:
                old_val = conv[i][j][0]
                conv[i][j][0] = strike
                #continue etc
                new_prices = {}
                new_prices[i] = []
                for k in range(i+1):
                    entry = conv[i][k]
                    new_prices[i].append(entry)
                #new_prices has one key i, and a list of length j
                new_prices = fill_payoff(new_prices, i)
                b_0 = new_prices[0][0][0]
                callable_prices.append(b_0)
                conv[i][j][0] = old_val
    return callable_prices

callable_prices_list = callable(cBond, K)
print("The possible callable is prices are:")
print(callable_prices_list)
print("The minnimum callable bond price is:")
print(min(callable_prices_list))

The possible callable is prices are:
[]
The minnimum callable bond price is:


ValueError: min() arg is an empty sequence

experiment code below

In [ ]:
# x is how much to change for additive model
def simulate_one(x):
    global vol
    v = x
    
    rates = r_values(0, 0.005)
    SP = StockPrice()
    
simulate_one(1)

In [ ]:
from scipy.stats import bernoulli

# rolls = {}

# for key in range(0, t+1, 1):
#         rolls[key] = [0]*(key+1)
#         for j in range(0, key+1, 1):
#             prob = max(0, min(1, p_vals[key][j]))
#             roll = bernoulli.rvs(size=1, p=prob)
#             rolls[key][j] = roll[0]


# if roll==1, then it is a head

interests = rates
for key in range(1, t+1, 1):
      for j in range(0, key+1, 1):
            prob = max(0, min(1, p_vals[key][j]))
            [roll] = bernoulli.rvs(size=1, p=prob)
            if(roll==1):
                  interests[key][j] = interests[key-1][j]+0.005
            else:
                  interests[key][j] = interests[key-1][j]-0.005


IndexError: index 1 is out of bounds for axis 0 with size 1